# All In One Executions

---

---

---

---

---

In [2]:
# Please fill in these values.
PROJECT_ID="development-set-or-th-mis"
LOCATION="us-central1"
REGION="us-central1"
SEARCH_ENGINE_ID="set_taxonomy"
DATA_STORE_LOCATION="global"
MAX_DOCUMENTS="5"
ENGINE_DATA_TYPE="1"

DATA_STORE_ID_PREFIX="set_taxonomy"
DATA_STORE_ID=f"{DATA_STORE_ID_PREFIX}_{PROJECT_ID}_01"
SEARCH_APP_ID="set_taxonomy_app"

import os

# Set environment variables
os.environ['PROJECT_ID'] = PROJECT_ID
os.environ['LOCATION'] = LOCATION
os.environ['REGION'] = REGION
os.environ['SEARCH_ENGINE_ID'] = SEARCH_ENGINE_ID
os.environ['DATA_STORE_LOCATION'] = DATA_STORE_LOCATION
os.environ['MAX_DOCUMENTS'] = MAX_DOCUMENTS
os.environ['ENGINE_DATA_TYPE'] = ENGINE_DATA_TYPE

os.environ['DATA_STORE_ID_PREFIX'] = DATA_STORE_ID_PREFIX
os.environ['DATA_STORE_ID'] = DATA_STORE_ID
os.environ['SEARCH_APP_ID'] = SEARCH_APP_ID

os.environ["LANGFUSE_SECRET_KEY"] = "sk-lf-03a6a329-97b3-4ac3-8a16-29afeb4b9543"
os.environ["LANGFUSE_PUBLIC_KEY"] = "pk-lf-68f1e305-bc5e-4dfe-b89c-edf24fb2d2c0"
os.environ["LANGFUSE_HOST"] = "https://cloud.langfuse.com"

```langfuse = Langfuse(
  secret_key="sk-lf-03a6a329-97b3-4ac3-8a16-29afeb4b9543",
  public_key="pk-lf-68f1e305-bc5e-4dfe-b89c-edf24fb2d2c0",
  host="https://cloud.langfuse.com"
)```

In [3]:
# Authenticate gcloud.
from google.colab import auth
auth.authenticate_user()

# Configure gcloud.
!gcloud config set project {PROJECT_ID}

ModuleNotFoundError: No module named 'google.colab'

In [5]:
import base64
import json
import pandas as pd
import requests
from pprint import pprint
from IPython.display import Image, display, Markdown

import vertexai
from vertexai.generative_models import GenerativeModel, Part, FinishReason, GenerationConfig
import vertexai.preview.generative_models as generative_models

from langchain_community.document_loaders import UnstructuredExcelLoader

import nltk
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

# Initialize Langfuse handler
from langfuse.decorators import observe, langfuse_context

from langfuse.callback import CallbackHandler
langfuse_handler = CallbackHandler(
    secret_key="sk-lf-03a6a329-97b3-4ac3-8a16-29afeb4b9543",
    public_key="pk-lf-68f1e305-bc5e-4dfe-b89c-edf24fb2d2c0",
    host="https://cloud.langfuse.com",
)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


In [6]:
# Vertex AI Init
vertexai.init(project="development-set-or-th-mis", location="us-central1")

generation_config = {
    "max_output_tokens": 8192,
    "temperature": 0.2,
    "top_p": 0.95,
}

safety_settings = {
    generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
}

model = GenerativeModel(
    "gemini-1.5-pro-001"
)

In [7]:
# System Instructions and Prompt Templates

textsi_taxonomy_mapping = """You are a helpful assistant to review the finacial reports and compare to taxonomy list.
Ensure your answers are complete, unless the user requests a more concise approach.
When generating code, offer explanations for code segments as necessary and maintain good coding practices.
When presented with inquiries seeking information, provide answers that reflect a deep understanding of the field, guaranteeing their correctness.
For any non-english queries, respond in the same language as the prompt unless otherwise specified by the user.
For prompts involving reasoning, provide a clear explanation of each step in the reasoning process before presenting the final answer."""


In [8]:
taxonomy_json_schema = {
    "type": "object",
    "properties": {
        "query": {
            "type": "string",
            "description": "The search query submitted by the user."
        },
        "set_code": {
            "type": "string",
            "description": "The SET Code corresponding to the identified item in the taxonomy."
        },
        "item_in_taxonomy": {
            "type": "string",
            "description": "The name of the item in the taxonomy that matches the query. Use NULL, if can not answer, or can not match"
        },
        "financial_statement_type": {
            "type": "string",
            "description": "The category in which item in the taxonomy belongs to"
        },
        "reasoning": {
            "type": "string",
            "description": "The reasoning behind the selection of the SET code and item in the taxonomy."
        }
    },
    "required": [
        "query",
        "set_code",
        "item_in_taxonomy",
        "reasoning"
    ]
}

In [9]:
# XLSX Document Loader Function
@observe()
def xlsx_document_loader(file_path: str):
    loader = UnstructuredExcelLoader(file_path, mode="single")
    docs = loader.load()

    print("XLSX Document parsed successfully.")

    return docs

- [\"สินทรัพย์\",\"หนี้สิน\",\"ส่วนของผู้ถือหุ้น\",\"หนี้สินและส่วนของผู้ถือหุ้น\",\"หนี้สินและส่วนของเจ้าของ\",\"ส่วนของผู้ถือหุ้น\",\"กระแสเงินสดจากกิจกรรมดำเนินงาน\",\"กระแสเงินสดจากกิจกรรมจัดหาเงิน\",\"กระแสเงินสดจากกิจกรรมลงทุน\"]


Example:
              - [\"งบการเงินที่แสดง เงินลงทุนตามวิธีส่วนได้เสีย พ.ศ. 2566 (บาท)\",\"งบการเงินที่แสดง เงินลงทุนตามวิธีส่วนได้เสีย พ.ศ. 2565 (บาท)\",\"งบการเงินที่แสดง งบการเงินเฉพาะกิจการ พ.ศ. 2566 (บาท)\",\"งบการเงินที่แสดง งบการเงินเฉพาะกิจการ พ.ศ. 2565 (บาท)\"]

In [10]:
@observe(as_type="generation")
def tables_extraction(parsed_xlsx: str):
    tables_schema = {
      "type": "ARRAY",
      "items": {
        "description": "Represents financial statement data.",
        "type": "OBJECT",
        "required": ["table_number", "table", "date", "headers", "currency_unit", "data"],
        "properties": {
          "table_number": {
            "description": "Sequence number of the table appeared from the source.",
            "type": "STRING"
          },
          "table": {
            "description": "The name of the financial statement.",
            "type": "STRING"
          },
          "date": {
            "description": "The date on which the financial statement is prepared.",
            "type": "STRING"
          },
          "headers": {
            "description": """Column headers for the financial data in single line format.

            """,
            "type": "ARRAY",
            "items": {
              "type": "STRING"
            }
          },
          "currency_unit": {
            "description": "The currency in which the financial data is expressed.",
            "type": "STRING"
          },
        }
      }
    }

    prompt_tables_extraction = f"""On a given financial report data extracted from an excel file as CONTEXT, extract the data by follow instructions.

<INSTRUCTIONS>
- You must use data only from CONTEXT
- Use original language from CONTEXT
- Extract all tables information to JSON format
</INSTRUCTIONS>

<CONTEXT>
{parsed_xlsx}
</CONTEXT>

Ouput in JSON:
"""

    input = [prompt_tables_extraction]

    response = model.generate_content(
        input,
        generation_config=GenerationConfig(
            max_output_tokens=8192,
            temperature=0.2,
            top_p=0.95,
            response_mime_type="application/json",
            response_schema=tables_schema,
            frequency_penalty=0.5,
        ),
        safety_settings=safety_settings,
        stream=False,
    )

    tables_json = json.loads(response.text)

    print("Found tables:")
    for table in tables_json:
        print(table.get("table_number"))
        print(table.get("table"))
        print(table.get("date"))
        print(table.get("headers"))
        print(table.get("currency_unit"))

    # Update langfuse observation
    input_count = model.count_tokens(str(input))
    output_count = model.count_tokens(response.candidates[0].content)
    langfuse_context.update_current_observation(
        input=str(input),
        output=str(response.text),
        usage={
            # usage
            "input": input_count.total_billable_characters,
            "output": output_count.total_billable_characters,
            #"total": ,  # if not set, it is derived from input + output
            "unit": "CHARACTERS",  # any of: "TOKENS", "CHARACTERS", "MILLISECONDS", "SECONDS", "IMAGES"
        },
        model_parameters={
            "temperature": 0.2,
            "top_p": 0.95,
            "max_output_tokens": 8192,
        },
        model="gemini-1.5-pro",
    )

    return json.loads(response.text)

In [11]:
@observe(as_type="generation")
def extract_table_data(table_info: str, parsed_xlsx: str):
    response_schema = {
      "type": "ARRAY",
      "items": {
        "description": "Represents financial statement data.",
        "type": "OBJECT",
        "required": ["table", "date", "headers", "currency_unit", "data"],
        "properties": {
          "table": {
            "description": "The name of the financial statement.",
            "type": "STRING"
          },
          "date": {
            "description": "The date on which the financial statement is prepared.",
            "type": "STRING"
          },
          "headers": {
            "description": "Column headers for the financial data.",
            "type": "ARRAY",
            "items": {
              "type": "STRING"
            }
          },
          "currency_unit": {
            "description": "The currency in which the financial data is expressed.",
            "type": "STRING"
          },
          "data": {
            "description": "The financial data itself.",
            "type": "ARRAY",
            "items": {
              "type": "OBJECT",
              "properties": {
                "category": {
                  "description": "The category or account name.",
                  "type": "STRING"
                },
                "items": {
                  "description": "Line items within a category.",
                  "type": "ARRAY",
                  "items": {
                    "type": "OBJECT",
                    "required": ["name", "values"],
                    "properties": {
                      "name": {
                        "description": "The name of the line item.",
                        "type": "STRING"
                      },
                      "note": {
                        "description": "Reference to a footnote for additional information.",
                        "type": "STRING"
                      },
                      "values": {
                        "description": "The financial values for each header.",
                        "type": "ARRAY",
                        "items": {
                          "type": "NUMBER"
                        }
                      }
                    }
                  }
                }
              }
            }
          }
        }
      }
    }

    prompt_with_schema = f"""On a given financial report data extracted from an excel file as CONTEXT, extract the data by follow instructions.

<INSTRUCTIONS>
- You must use data only from CONTEXT
- Use original language from CONTEXT
- Extract the table {table_info} data to JSON format
</INSTRUCTIONS>

<CONTEXT>
{parsed_xlsx}
</CONTEXT>

Ouput in JSON:
"""

    input = [prompt_with_schema]

    response = model.generate_content(
        input,
        generation_config=GenerationConfig(
            max_output_tokens=8192,
            temperature=0.2,
            top_p=0.95,
            response_mime_type="application/json",
            response_schema=response_schema
        ),
        safety_settings={
            generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
            generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
            generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
            generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        },
        stream=False,
    )

    # Update langfuse observation
    input_count = model.count_tokens(str(input))
    output_count = model.count_tokens(response.candidates[0].content)
    langfuse_context.update_current_observation(
        input=str(input),
        output=str(response.text),
        usage={
            # usage
            "input": input_count.total_billable_characters,
            "output": output_count.total_billable_characters,
            #"total": ,  # if not set, it is derived from input + output
            "unit": "CHARACTERS",  # any of: "TOKENS", "CHARACTERS", "MILLISECONDS", "SECONDS", "IMAGES"
        },
        model_parameters={
            "temperature": 0.2,
            "top_p": 0.95,
            "max_output_tokens": 8192,
        },
        model="gemini-1.5-pro",
    )

    return response

In [12]:
def response_info(response):
    # Response tokens count
    usage_metadata = response.usage_metadata
    print(f"Prompt Token Count: {usage_metadata.prompt_token_count}")
    print(f"Candidates Token Count: {usage_metadata.candidates_token_count}")
    print(f"Total Token Count: {usage_metadata.total_token_count}")
    print(response.candidates[0])
    print(response.text)

In [13]:
def llm_parser_xlsx_to_json(tables_json: list, parsed_xlsx: str):
    data_json = []
    for table in tables_json:
        print(f"Parsing table: {table.get('table_number')}: {table.get('table')}")
        response = extract_table_data(
            table_info=table,
            parsed_xlsx=parsed_xlsx,
        )
        data_json.append(json.loads(response.text))

    display_json = [ [x['table'], y['category'], z['name'], z['values']] for x in [ x[0] for x in data_json ] for y in x['data'] for z in y['items'] ]
    return display_json

In [14]:
@observe()
def xlsx_extraction_run(file_path: str):
    # Parse XLSX to Document
    doc = xlsx_document_loader(file_path)

    # Extract Tables list information
    tables_json = tables_extraction(
        parsed_xlsx=doc[0].page_content,
    )

    # Extract Tables data
    data_json = llm_parser_xlsx_to_json(
        tables_json=tables_json,
        parsed_xlsx=doc[0].page_content,
    )

    return data_json

In [15]:
@observe(as_type="generation")
def generate(
    input,
    generation_config: dict = None,
    safety_settings: dict = None,
    system_instruction: str = None,
    model: str = "gemini-1.5-pro-001",
    metadata: str = "NULL",
):
    model = GenerativeModel(
        model,
        system_instruction=[system_instruction]
    )
    response = model.generate_content(
        input,
        generation_config=GenerationConfig(
            max_output_tokens=8192,
            temperature=0.2,
            top_p=0.95,
            response_mime_type="application/json",
            response_schema=taxonomy_json_schema
        ),
        safety_settings=safety_settings,
        stream=False,
    )

    # print(response.text, end="")

    # Update langfuse observation
    input_count = model.count_tokens(str(input))
    output_count = model.count_tokens(response.candidates[0].content)
    langfuse_context.update_current_observation(
        input=str(input)+('|')+metadata,
        output=str(response.text),
        usage={
            # usage
            "input": input_count.total_billable_characters,
            "output": output_count.total_billable_characters,
            #"total": ,  # if not set, it is derived from input + output
            "unit": "CHARACTERS",  # any of: "TOKENS", "CHARACTERS", "MILLISECONDS", "SECONDS", "IMAGES"
        },
        model_parameters={
            "temperature": 0.2,
            "top_p": 0.95,
            "max_output_tokens": 8192,
        },
        model="gemini-1.5-pro",
    )

    return response

In [16]:
@observe()
def search_structured_datastore(query: str) -> str:
    """Retrive the information of Taxomony list that help to identify SET Code from input query"""
    from langchain_community.retrievers import (
        GoogleVertexAISearchRetriever,
    )
    import json

    retriever = GoogleVertexAISearchRetriever(
        project_id=PROJECT_ID,
        search_engine_id=SEARCH_APP_ID,
        location_id=DATA_STORE_LOCATION,
        engine_data_type=1,
        max_documents=10,
    )
    results = retriever.invoke(query)

    docs = ""
    if len(results) == 0:
        return "Taxonomy Information Not Found. Please apologize and say I could not answer"
    for result in results:
        d = json.loads(result.page_content)
        docs += f"{d}\n"
        #docs += f"SET Code: {d.get('set_code')}\nItem in taxonomy: {d.get('item_in_taxonomy')}\nMeaning of taxonomy: {d.get('meaning_of_taxonomy')}\n\n"

    return docs

In [17]:
@observe()
def taxonomy_mapping(
    query: str,
    model: str = "gemini-1.5-pro-001",
    metadata: str = "NULL",
    financial_statement_type: str = ""
) -> str:
    """Retrive the information of Taxomony list that help to identify SET Code from input query"""

    # Search with query in Vertex AI Structured Datastore
    search_result = search_structured_datastore(query=query)

    # Prepare input for Gemini
    input = [
        "From the financial report line item name as INPUT. Use the Taxonomy list in CONTEXT and follow INSTRUCTION to suggest the correct SET Code and Item in Taxonomy Name.",
        f"""<CONTEXT>{search_result}</CONTEXT>""",
        f"<INSTRUCTIONS>the value of property financial_statement_type in suggested output must be equal or similar to {financial_statement_type}</INSTRUCTIONS>",
        f"INPUT: {query}",
        "OUTPUT IN JSON (query, set_code, item_in_taxonomy, financial_statement_type) and Reasoning:"
    ]

    # Generate response from Gemini
    response = generate(
        input,
        generation_config,
        safety_settings,
        textsi_taxonomy_mapping,
        model,
        metadata
    )

    return search_result, response.text

In [18]:
@observe()
def taxonomy_mapping_run(
    data_json: list,
    model: str = "gemini-1.5-pro-001",
):
    data_json_with_taxonomy = []

    for data in data_json:
        # Set Metadata to Table name, Category, and Number values
        metadata = data[0]+"-"+data[1]+"-"+str(data[3])

        financial_statement_type = data[0]
        fmt_fs_type = ''
        if ("งบฐานะการเงิน" in financial_statement_type):
          fmt_fs_type = 'BS'
        elif ("งบกำไรขาดทุน" in financial_statement_type):
          fmt_fs_type = 'PL'
        elif ("งบกระแสเงินสด" in financial_statement_type):
          fmt_fs_type = 'CF'

        if (fmt_fs_type != ''):
          # Gemini + Vertex AI Search Datastore Retriver
          predicted_taxonomy = taxonomy_mapping(query=data[2], model=model, metadata=metadata, financial_statement_type=fmt_fs_type)

          # Parse result string to JSON
          data_json = json.loads(predicted_taxonomy[1])

          # Append to list
          data_json_with_taxonomy.append(
              {
                  "query": data_json.get('query'),
                  "set_code": data_json.get('set_code'),
                  "item_in_taxonomy": data_json.get('item_in_taxonomy'),
                  "reasoning": data_json.get('reasoning'),
                  "table_name": data[0],
                  "category": data[1],
                  "number_values": str(data[3]),
              }
          )
          print("Input Query: " + data[2] + "\nPredicted: " + predicted_taxonomy[1])

    return data_json_with_taxonomy

In [19]:
@observe(capture_input=False, capture_output=False)
def app_run(
    gcs_uri: str,
    model: str = "gemini-1.5-pro-001",
):
    file_path = gcs_uri

    # Extract XLSX file with Langchain Unstructured.io
    data_json = xlsx_extraction_run(
        file_path=file_path,
    )

    # Gemini run get tables information and number of tables, then run taxonomy mapping
    data_json_with_taxonomy = taxonomy_mapping_run(
        data_json=data_json,
        model=model,
    )

    langfuse_context.update_current_observation(
        input=gcs_uri, # any serializable object
        output=str(data_json_with_taxonomy), # any serializable object
    )

    return pd.DataFrame(data_json_with_taxonomy)

In [20]:
run_df = app_run(
    gcs_uri="./SCC_FINANCIAL_STATEMENTS.XLSX",
    model="gemini-1.5-pro-001",
)

XLSX Document parsed successfully.
Found tables:
1
งบฐานะการเงินรวม
30 มิถุนายน 2567
['หมายเหตุ', '30 มิถุนายน 2567', '31 ธันวาคม 2566']
พันบาท
2
งบฐานะการเงินรวม
30 มิถุนายน 2567
['หมายเหตุ', '30 มิถุนายน 2567', '31 ธันวาคม 2566']
พันบาท
3
งบฐานะการเงินรวม
30 มิถุนายน 2567
['30 มิถุนายน 2567', '31 ธันวาคม 2566']
พันบาท
4
งบกำไรขาดทุนรวม
30 มิถุนายน 2567
['2567', '2566.0']
พันบาท
5
งบกำไรขาดทุนเบ็ดเสร็จรวม
30 มิถุนายน 2567
['2567', '2566.0']
พันบาท
6
งบกำไรขาดทุนรวม
30 มิถุนายน 2567
['2567', '2566.0']
พันบาท
7
งบกำไรขาดทุนเบ็ดเสร็จรวม
30 มิถุนายน 2567
['2567', '2566.0']
พันบาท
8
งบการเปลี่ยนแปลงส่วนของผู้ถือหุ้นรวม
30 มิถุนายน 2567
['หมายเหตุ', 'รวมส่วนของผู้ถือหุ้นของบริษัท', 'ส่วนได้เสียที่ไม่มีอำนาจควบคุม', 'รวมส่วนของผู้ถือหุ้น']
พันบาท
9
งบการเปลี่ยนแปลงส่วนของผู้ถือหุ้นรวม
30 มิถุนายน 2567
['หมายเหตุ', 'รวมส่วนของผู้ถือหุ้นของบริษัท', 'ส่วนได้เสียที่ไม่มีอำนาจควบคุม', 'รวมส่วนของผู้ถือหุ้น']
พันบาท
10
งบกระแสเงินสดรวม
30 มิถุนายน 2567
['2567', '2566.0']
พันบาท
11
งบกระแสเงินสดรวม

KeyboardInterrupt: 

In [ ]:
# with open('./scc_data.json', 'w') as scc_file:
#       scc_file.write(str(data_json))

In [ ]:
run_df

,query,set_code,item_in_taxonomy,reasoning,table_name,category,number_values
0,สินทรัพย์หมุนเวียน,110000,สินทรัพย์หมุนเวียน,The query directly matches the item 'สินทรัพย์...,งบฐานะการเงินรวม,สินทรัพย์,"[252196240, 217214488]"
1,สินทรัพย์ไม่หมุนเวียน,120000,สินทรัพย์ไม่หมุนเวียน,The query directly matches the item 'สินทรัพย์...,งบฐานะการเงินรวม,สินทรัพย์,"[697281874, 676386144]"
2,รวมสินทรัพย์,109999,รวมสินทรัพย์,Found an exact match for the input query 'รวมส...,งบฐานะการเงินรวม,สินทรัพย์,"[949478114, 893600632]"
3,หนี้สินหมุนเวียน,210000,หนี้สินหมุนเวียน,"The query directly matches the item ""หนี้สินหม...",งบฐานะการเงินรวม,หนี้สินและส่วนของผู้ถือหุ้น,"[234656533, 202244839]"
4,หนี้สินไม่หมุนเวียน,220000,หนี้สินไม่หมุนเวียน,"The query directly matches the item ""หนี้สินไม...",งบฐานะการเงินรวม,หนี้สินและส่วนของผู้ถือหุ้น,"[259404357, 249759153]"
...,...,...,...,...,...,...,...
282,จ่ายภาษีเงินได้สำหรับงวด,NULL,NULL,"I apologize, I do not have access to the taxon...",งบกระแสเงินสดรวม,ข้อมูลงบกระแสเงินสดเปิดเผยเพิ่มเติม,"[-3182569, -2576441.0]"
283,เงินปันผลค้างรับ,NULL,NULL,There is no item in the taxonomy that matches ...,งบกระแสเงินสดรวม,ข้อมูลงบกระแสเงินสดเปิดเผยเพิ่มเติม,"[1876655, 1575225.0]"
284,เจ้าหนี้จากการซื้อสินทรัพย์,NULL,NULL,"I apologize, I do not have access to the Taxon...",งบกระแสเงินสดรวม,ข้อมูลงบกระแสเงินสดเปิดเผยเพิ่มเติม,"[2098408, 6641963.0]"
285,เงินลงทุนค้างจ่าย,NULL,NULL,"I apologize, I do not have access to the taxon...",งบกระแสเงินสดรวม,ข้อมูลงบกระแสเงินสดเปิดเผยเพิ่มเติม,"[578257, 538921.0]"


In [ ]:
run_df.to_excel('SCC_mapping_result.xlsx')
print("Write Success")

Write Success


In [ ]:
%%shell
pwd

/content


In [ ]:
import gradio as gr

with gr.Blocks() as demo:
    input_text = gr.Textbox(label="XLSX File")

    with gr.Row():
        output_df = gr.DataFrame(label="Output DataFrame")

    submit_btn = gr.Button("Map Taxonomy")
    submit_btn.click(
        fn=app_run,
        inputs=[input_text],
        outputs=[output_df],
    )

demo.launch(debug=True,)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://ae6fc381c6a3653fd8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


XLSX Document parsed successfully.
Found tables:
1
งบแสดงฐานะการเงิน
ณ วันที่ 31 ธันวาคม พ.ศ. 2566
['พ.ศ. 2566', 'พ.ศ. 2565', 'พ.ศ. 2566', 'พ.ศ. 2565']
บาท
2
งบแสดงฐานะการเงิน
ณ วันที่ 31 ธันวาคม พ.ศ. 2566
['พ.ศ. 2566', 'พ.ศ. 2565', 'พ.ศ. 2566', 'พ.ศ. 2565']
บาท
3
งบแสดงฐานะการเงิน
ณ วันที่ 31 ธันวาคม พ.ศ. 2566
['พ.ศ. 2566', 'พ.ศ. 2565', 'พ.ศ. 2566', 'พ.ศ. 2565']
บาท
4
งบกำไรขาดทุนเบ็ดเสร็จ
สำหรับปีสิ้นสุดวันที่ 31 ธันวาคม พ.ศ. 2566
['พ.ศ. 2566', 'พ.ศ. 2565', 'พ.ศ. 2566', 'พ.ศ. 2565']
บาท
5
งบกำไรขาดทุนเบ็ดเสร็จ
สำหรับปีสิ้นสุดวันที่ 31 ธันวาคม พ.ศ. 2566
['พ.ศ. 2566', 'พ.ศ. 2565', 'พ.ศ. 2566', 'พ.ศ. 2565']
บาท
6
งบแสดงการเปลี่ยนแปลงส่วนของเจ้าของ
สำหรับปีสิ้นสุดวันที่ 31 ธันวาคม พ.ศ. 2566
['หมายเหตุ', 'บาท', 'บาท', 'บาท', 'บาท']
บาท
7
งบแสดงการเปลี่ยนแปลงส่วนของเจ้าของ
สำหรับปีสิ้นสุดวันที่ 31 ธันวาคม พ.ศ. 2566
['หมายเหตุ', 'บาท', 'บาท', 'บาท', 'บาท']
บาท
8
งบกระแสเงินสด
สำหรับปีสิ้นสุดวันที่ 31 ธันวาคม พ.ศ. 2566
['พ.ศ. 2566', 'พ.ศ. 2565', 'พ.ศ. 2566', 'พ.ศ. 2565']
บาท
9
งบกระแสเงินสด
